In [7]:
from __future__ import print_function
%matplotlib tk
from ReductionImports import *
import  NicoleNative as nic
setrcParams()
INITDIR = getFileFilesDir(2)
print(INITDIR)

/home/hp/NICOLE/run


In [33]:
wav_start = 8537.692427
wav_samp = 0.010585
wav_n = 60
wav_ran = arange(wav_n)*wav_samp + wav_start
# For synthesis mode
model_in = 'mymodel.model'
model_interp = 'fei-8538-interp.mod'
profile_synth = 'fei-8538-synth.pro'
# For inversion mode
model_guess = 'mymodel.model'
model_out = 'fei-8538-fitted.mod'
profile_obs = 'fei-8538-obs.pro'
profile_out = 'fei-8538-fitted.pro'
# New model
me_pars, me_head, me_const, me_out = nic.read_ascii_model('holmu.model')
# me_const = ' ' + str(int((wav_samp*0/8538)*2.998*10**10)) + ' 0.0\r\n'
N = me_out.shape[0]
me_out[:,4] = linspace(1000, 5000, N) # b_los
me_out[:,5] = linspace(200, 2000, N) # v_los
me_out[:,6] = linspace(50, 500, N) # b_x
me_out[:,7] = linspace(50, 500, N) # b_y
nic.create_ascii_model(model_in, me_head, me_const, me_out)

In [35]:
INPUT = nic.set_nicole_input_params(Mode = 'S',
                                Input_model = model_in ,
                                Output_profiles = profile_synth,
                                Heliocentric_angle = 0.9, 
                                Output_model = model_interp,
                                Continuum_reference = 4,
                                Debug_mode = 1)
INPUT = nic.set_nicole_regions_lines(INPUT, 'Region 1',
                                 First_wavelength = wav_start, 
                                 Wavelength_step = wav_samp,
                                 Number_of_wavelengths = wav_n)
INPUT = nic.set_nicole_regions_lines(INPUT, 'Line 1',
                                 Line = 'FeI_8538')
F = open('NICOLE.input', 'w')
F.write(INPUT)
F.close()
try:
    os.remove(profile_synth)
except: pass

In [36]:
%run '/home/hp/NICOLE/run/run_nicole.py'
close()

Checking syntax in file:LINES
Preparing cycle 1
Checking syntax in file:NICOLE.input
  ... no errors found

Preparing input model...100%

Starting code execution


In [37]:
# mi_pars, mi_head, mi_const, mi_plot = nic.read_ascii_model(model_in)
# mt_pars, mt_head, mt_plot, mt_const, mt_abu = nic.read_native_model(model_interp)
# nic.plot_model_bvtp(figure(1), [mi_plot, mt_plot])
s_synth = nic.read_native_profile([profile_synth])
nic.plot_profile(figure(2), s_synth)

nicole2.3bp     
1 1 60


ValueError: cannot reshape array of size 0 into shape (4,60,1,1)

In [126]:
[1,2]+[3]

[1, 2, 3]

# Add noise and invert

In [129]:
wavs = reshape(wav_ran, (1, wav_n))
stokes = s_synth[0][:,:,0,0]
for i in range(4):
    stokes[1::,:] = stokes[1::,:] + 0.001*random.randn(3,wav_n)*stokes[0,:]
pro_obs = append(wavs , stokes, axis=0)
savetxt(profile_obs, transpose(pro_obs), fmt='{:^10}'.format('%s'), delimiter=' ')

In [130]:
INPUT = nic.set_nicole_input_params(Mode = 'I',
                                Input_model = model_guess,
                                Observed_profiles = profile_obs,
                                Output_profiles = profile_out,
                                Heliocentric_angle = 0.9, 
                                Output_model = model_out,
                                noise_level = 0.001,
                                Maximum_number_of_inversions=10,
                                Continuum_reference = 4,
                                Debug_mode = 1)
INPUT = nic.set_nicole_regions_lines(INPUT, 'Region 1',
                                 First_wavelength = wav_start, 
                                 Wavelength_step = wav_samp,
                                 Number_of_wavelengths = wav_n)
INPUT = nic.set_nicole_regions_lines(INPUT, 'Line 1',
                                 Line = 'FeI_8538')
F = open('NICOLE.input', 'w')
F.write(INPUT)
F.close()
try:
    os.remove(profile_out)
    os.remove(model_out)
except: pass

In [132]:
mo_pars, mo_head, mo_plot, mo_const, mo_abu = nic.read_native_model(model_out)
nic.plot_model_bvtp(figure(3), [mi_plot, mo_plot])
s_synth_out = nic.read_native_profile([profile_synth, profile_out])
nic.plot_profile(figure(4), s_synth_out + [stokes])

nicole2.3bp     
1 1 60
nicole2.3bp     
1 1 60
